In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
from time import sleep
import yagmail

#abrir o navegador
servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)

tabela = pd.read_excel('buscas.xlsx')
display(tabela)

def busca_google_shopping(navegador,produto,termos_banidos,preco_min,preco_max):
    navegador.get('https://www.google.com.br/')
    
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(' ')
    lista_termos_produto = produto.split(' ')
    preco_max = float(preco_max)
    preco_min = float(preco_min)

    navegador.find_element(By.TAG_NAME,'input').send_keys(produto,Keys.ENTER)
    navegador.find_element(By.XPATH,'//*[@id="hdtb-msb"]/div[1]/div/div[2]/a').click()

    lista_resultados = navegador.find_elements(By.CLASS_NAME,'sh-dgr__grid-result')
    lista_ofertas = []
    for resultado in lista_resultados:
        nome = resultado.find_element(By.CLASS_NAME,'Xjkr3b').text
        nome = nome.lower()
        #verificação do nome pesquisado
        tem_termos_banidos = False
        for palavra in lista_termos_banidos:
            if palavra in nome:
                tem_termos_banidos = True

        tem_todos_termos_produto = True
        for palavra in lista_termos_produto:
            if palavra not in nome:
                tem_todos_termos_produto = False

        if not tem_termos_banidos and tem_todos_termos_produto:
            try:
                preco = resultado.find_element(By.CLASS_NAME,'a8Pemb').text
                preco = preco.replace('R$','').replace(' ','').replace('.','').replace(',','.')
                preco = float(preco)
            
                if preco_min <= preco <= preco_max:
                    elemento_link = resultado.find_element(By.CLASS_NAME,'aULzUe')
                    elemento_pai = elemento_link.find_element(By.XPATH,'..')
                    link = elemento_pai.get_attribute('href')
                    lista_ofertas.append((nome,preco,link))
            except:
                continue
    return lista_ofertas               

def busca_buscape(navegador,produto,termos_banidos,preco_min,preco_max):
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(' ')
    lista_termos_produto = produto.split(' ')
    preco_max = float(preco_max)
    preco_min = float(preco_min)
    
    navegador.get('https://www.buscape.com.br/')
    navegador.find_element(By.CLASS_NAME,'AutoCompleteStyle_input__FInnF').send_keys(produto,Keys.ENTER)
    
    sleep(5)
    lista_resultados = navegador.find_elements(By.CLASS_NAME,'Cell_Content__fT5st')
    lista_ofertas = []
    for resultado in lista_resultados:
        try:
            nome = resultado.get_attribute('title')
            nome = nome.lower()
            preco = resultado.find_element(By.CLASS_NAME,'CellPrice_MainValue__JXsj_').text
            link = resultado.get_attribute('href')

            tem_termos_banidos = False
            for palavra in lista_termos_banidos:
                if palavra in nome:
                    tem_termos_banidos = True        

            tem_todos_termos_produto = True
            for palavra in lista_termos_produto:
                if palavra not in nome:
                    tem_todos_termos_produto = False

            if not tem_termos_banidos and tem_todos_termos_produto:
                preco = preco.replace('R$','').replace(' ','').replace('.','').replace(',','.')
                preco = float(preco)
                
                if preco_min <= preco <= preco_max:
                    lista_ofertas.append((nome,preco,link))
        except:
            pass
        
    return lista_ofertas                        

tabela_ofertas = pd.DataFrame()
for linha in tabela.index:    
    produto = tabela.loc[linha,'Nome']
    termos_banidos = tabela.loc[linha,'Termos banidos']
    preco_min = tabela.loc[linha,'Preço mínimo']
    preco_max = tabela.loc[linha,'Preço máximo']

    lista_ofertas_google_shopping = busca_google_shopping(navegador,produto,termos_banidos,preco_min,preco_max)
    if lista_ofertas_google_shopping:
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shopping,columns=['Produto','Preço','Link'])
        tabela_ofertas = tabela_ofertas.append(tabela_google_shopping)
    else:
        tabela_google_shopping = None
        
    lista_ofertas_buscape = busca_buscape(navegador,produto,termos_banidos,preco_min,preco_max)
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape,columns=['Produto','Preço','Link'])
        tabela_ofertas = tabela_ofertas.append(tabela_buscape)
    else:
        tabela_buscape = None
    
display(tabela_ofertas)

tabela_ofertas = tabela_ofertas.reset_index(drop=True)
tabela_ofertas.to_excel('Ofertas.xlsx',index=False)

corpo_email = f''' 
<p>Prezados,</p>
<p>Encontramos alguns produtos em oferta dentro da faixa de preço desejada. Segue tabela com detalhes</p>
{tabela_ofertas.to_html(index=False)}
<p>Qualquer dúvida estou à disposição</p>
<p>Att.,</p>
'''
corpo_email = corpo_email.replace('\n', ' ')
if len(tabela_ofertas.index) > 0:
    usuario = yagmail.SMTP(user='SeuEmail@gmail.com', password='xxxxxxxx')
    usuario.send(to='EmaildeDestino@gmail.com',
                 subject='Produto(s) Encontrado(s) na faixa de preço desejada',
                 contents=corpo_email)
navegador.quit()    